# Continuation of my datascraping file NLP_lab1_amsiejus.py

In [1]:
# Connect google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas
# reading the CSV file
csvFile = pandas.read_csv('/content/gdrive/MyDrive/Studijos/4 kursas/Naturalios kalbos apdorojimas/houzz_final_data.csv')
 
# displaying the contents of the CSV file
csvFile.head()

,title,description,price
0,10 Inch Dark Brown Elmwood Chinese Square Stand,"Made of Elmwood, this square stand compliments...",$63
1,"12"" Crystal End Table, Clear Mirror Finish",The modern collection is all over the crystal ...,$165
2,"13""x22""x23"" White Wood Veneer Side Table",The compact profile of this magnificent side t...,$153
3,"149 RAF 2 Piece Sectional Set, Ariel Slate",Picket House Furnishings Maddox Right Arm Faci...,"$4,158"
4,"16"" Modern Geometric Square Side Table, Dark W...",Break the mold of boring decor with this geome...,$118


## Semantic Textual Similarity for e-shops
[Example page](https://www.sbert.net/docs/usage/semantic_textual_similarity.html?fbclid=IwAR0NUILKOkBLuKgRXTtcjjsyozwrJ8ihQapF0kbsIlUZ06RC8ZkDEReU5G8) from SBert.net

In [3]:
request_sentence = 'Black sectional sofa with ottoman and chaise'
database_sentences = csvFile['description']

In [4]:
!pip install sentence_transformers

Sentance trasformer (can do all) [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

### Easy way (automatically tokenise, compute token embeddings, perform pooling and normalize sentences)

In [6]:
# Embedding the scraped dataframe
embeddings = model.encode(database_sentences, convert_to_tensor=True)
embeddings
# Yes, that is it

tensor([[ 0.0412,  0.0867,  0.0244,  ..., -0.1151, -0.0070,  0.0750],
        [-0.0229, -0.0153,  0.0240,  ..., -0.0437,  0.0188, -0.0042],
        [ 0.0439,  0.0358, -0.0486,  ...,  0.0275, -0.0705,  0.0623],
        ...,
        [-0.0195,  0.0350,  0.0048,  ...,  0.0704, -0.0711,  0.0064],
        [-0.0195,  0.0350,  0.0048,  ...,  0.0704, -0.0711,  0.0064],
        [ 0.1178,  0.0228,  0.0542,  ...,  0.0090, -0.0306, -0.0151]],
       device='cuda:0')

### DIY embedding (**DOES NOT WORK**. Left only to determine and fix my error)

Won't put code here since for some reason it crashes due to lack of RAM. You can  find og code in the link above.

In [7]:
test = list(database_sentences)

In [8]:
import torch
from transformers import AutoModel, AutoTokenizer
# if using colab/mif cloud/GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
# 1a) Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [10]:
model_manual = AutoModel.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [11]:
tokenized_list = []

for description in test:
  tokenized_list.append(tokenizer.encode(description, padding=True, truncation=True, return_tensors='pt').to(device))

Crashes after lack of memory. This part was left only to ask question later

In [ ]:
embedded = []
for tokenized in tokenized_list:
  embedded.append(torch.mean(model_manual(tokenized).last_hidden_state, axis = 1))

## Search function:

In [9]:
def getSimilarSentence(request_sentence, df, df_embedded):
  # Embed the model
  embedding = model.encode(request_sentence, convert_to_tensor=True)
  # Get cosine similarity scores (array)
  cosine_scores = util.cos_sim(embedding, df_embedded)

  # Assign to new column in og dataframe (if it doesn't work try deleting cpu() since I was using gpu for this)
  df['cosine scores'] = cosine_scores.cpu().numpy()[0]

  top_matches = df.sort_values("cosine scores", ascending=False).head(3)
  print(top_matches)

In [10]:
getSimilarSentence(request_sentence, csvFile, embeddings)

                                                  title  \
218   ACME Ceasar Sectional Sofa, Rev. Ottoman, Gray...   
328   Albi 3 Piece Sectional Sofa Set Covered in Cit...   
1638  GDF Studio Bridger Fabric Sectional Couch With...   

                                            description    price  \
218   Create a warm and inviting style in your home ...  $1,528    
328   This 3-piece sectional sofa covered in a smoot...  $1,483    
1638  This sectional couch is a great addition for y...    $834    

      cosine scores  
218        0.767138  
328        0.756782  
1638       0.754998  


In [ ]:
print(database_sentences[218],"\n")
print(database_sentences[328],"\n")
print(database_sentences[1638],"\n")

Create a warm and inviting style in your home with the Caesar sectional sofa and ottoman. This gray upholstered sofa and reversible ottoman will bring you a modern and comfortable sensation in the living room. Featuring track arms, loose back and seat cushion, squared legs, and multi-function ottoman that can transform the sofa into a sectional or chair into chaise. This set will present you a better and soft environment with family and friends. Some Assembly Required.Sectional Sofa & Reversible Ottoman, Multi-Function Ottoman transforms Sofa into Sectional or Chair into Chaise, Loose Back & Seat Cushion, Seat w/Pocket Coil Included, Track Arm Plastic Leg, KD Arm/Back/Leg 

This 3-piece sectional sofa covered in a smooth but functional fabricLower portion is trimmed with nickel finished studsThe seating is of pure, plush comfort.This platform framed sectional also features short wooden legs, accent stitchingWith matching ottoman.Available in Citrus/Orane PolyfiberActual product color m

Other examples:

In [ ]:
# Product descriptions from amazon:
sentence1 = 'VASAGLE End Tables Set of 2, Side Tables with Storage Shelf, Slim Night Tables, Steel Frame, for Living Room, Study, Bedroom, Industrial, Rustic Brown and Black'
sentence2 = 'Casamudo TV Stand Media Console, Mid-Century Modern Television Table with Cabinet Doors, Wooden Low-Profile Media Furniture for TVs up to 78 inches with Splayed Legs'
sentence3 = 'Amazon Brand – Rivet Modern Living Room Accent Chair with Metal Legs, 35.4"W, Dark Gray'

In [ ]:
getSimilarSentence(sentence1, csvFile, embeddings)

                                                  title  \
3818         Yarlow Black Occasional 3-Piece Table, Set   
1537  Furniture of America Deldrin Wood 1-Shelf Side...   
1558  Furniture of America Humere Wood 1-Shelf End T...   
1548  Furniture of America Eva Contemporary Metal 1-...   
227   ACME Enca 2 Pieces Nesting Table in Clear Glas...   

                                            description  price  cosine scores  
3818  This occassional table set offers casual, cont...  $251        0.723022  
1537  Create the perfect place for decor like framed...  $350        0.705916  
1558  Embrace casual, welcoming style in your living...  $660        0.704411  
1548  Add some pizzazz to your living room décor wit...  $112        0.692573  
227   Create double the end table space with the con...  $333        0.681382  


In [ ]:
print(database_sentences[3818],"\n")
print(database_sentences[1537],"\n")
print(database_sentences[1558],"\n")

This occassional table set offers casual, contemporary design for modern living. With a functional nesting design, this set provides a cocktail table and two end tables with storage cubbies that slide together for small space living solutions. These pieces have a replicated blackened wood finish with a grayed, textural grain for a classic look. The metal bases elevate the table to create open space that adds a sense of lightness to your room. 

Create the perfect place for decor like framed photos and sculpture with this set of two transitional wood side tables. Each shows off a dimensional finish complementing a lower shelf and tapered legs for a clean, appealing presentation. The tables in blue finish are a charming addition to your living room seating ensemble, offering a casual, inviting ambiance you're going to love. Choose a selection of books to add a personalized touch.Features :RELIABLE TABLE TOP MATERIAL - Their tops are formed by acacia wood, birch veneer and wood veneer tha